## Preparing the data

In [1]:
import pandas as pd
import numpy

sdg = pd.read_csv("C:\\Users\\Jiachen\\Downloads\\SDG Database UN\\data\\simplifiedGOAL1-17.csv",encoding='utf-8')
#In Excel, remove non-numeric characters in Value (e.g. >2.5  will be 2.5)

#Drop rows where Value or TimePeriod is missing
sdg.dropna(subset=['Value'], axis = 0, inplace=True) 
sdg.dropna(subset=['TimePeriod'], axis = 0, inplace=True) 
#change data type of the columns
sdg.TimePeriod=sdg.TimePeriod.astype(int)
sdg.Goal=sdg.Goal.astype(str)
sdg.GeoAreaCode =sdg.GeoAreaCode .astype(str)

sdg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1047726 entries, 2 to 1048568
Data columns (total 10 columns):
Goal                 1047726 non-null object
Target               1047726 non-null object
Indicator            1047726 non-null object
SeriesCode           1047726 non-null object
SeriesDescription    1047726 non-null object
GeoAreaCode          1047726 non-null object
GeoAreaName          1047726 non-null object
TimePeriod           1047726 non-null int32
Value                1047726 non-null float64
Units                1047726 non-null object
dtypes: float64(1), int32(1), object(8)
memory usage: 83.9+ MB


In [2]:
#hierarchical dataframe Indicator_SeriesCode_GeoAreaName (isg) 
isg=sdg.set_index(['Goal','Target','Indicator','SeriesCode','GeoAreaName'],drop=True)
#pivot table where the columns are the years
isg=isg.pivot_table(index=['Goal','Target','Indicator','SeriesCode','GeoAreaName'], columns='TimePeriod', values='Value')

#isg.info()   #columns 2005-2016 have >20000 data entries
    #subset columns 2005-2016
isg=isg[isg.columns[5:17]]

#count the null values in each row and output to a list
null_count=isg.notnull().sum(axis=1).tolist()

from numpy import array
isg['NotNull']= array(null_count) / len(isg.columns)  #divide null_count by 12
isg.head(10)

#add 2 columns
isg['2025'] = numpy.nan
isg['2030'] = numpy.nan
isg.head()

TimePeriod                                     2005  2006  2007  2008  2009  \
Goal Target Indicator SeriesCode  GeoAreaName                                 
1    1.1    1.1.1     SI_POV_DAY1 Albania       1.1   NaN   NaN   0.4   NaN   
                                  Algeria       NaN   NaN   NaN   NaN   NaN   
                                  Angola        NaN   NaN   NaN  30.1   NaN   
                                  Argentina     3.9   3.3   2.9   2.6   2.6   
                                  Armenia       4.4   3.2   2.8   1.4   1.9   

TimePeriod                                     2010  2011  2012  2013  2014  \
Goal Target Indicator SeriesCode  GeoAreaName                                 
1    1.1    1.1.1     SI_POV_DAY1 Albania       NaN   NaN   1.1   NaN   NaN   
                                  Algeria       NaN   0.5   NaN   NaN   NaN   
                                  Angola        NaN   NaN   NaN   NaN   NaN   
                                  Argentina     1.1   0.9   0.8   0.8   0.7   
                                  Armenia       1.9   2.2   1.6   2.2   2.3   

TimePeriod                                     2015  2016   NotNull  2025  \
Goal Target Indicator SeriesCode  GeoAreaName                               
1    1.1    1.1.1     SI_POV_DAY1 Albania       NaN   NaN  0.250000   NaN   
                                  Algeria       NaN   NaN  0.083333   NaN   
                                  Angola        NaN   NaN  0.083333   NaN   
                                  Argentina     NaN   0.6  0.916667   NaN   
                                  Armenia       1.9   1.8  1.000000   NaN   

TimePeriod                                     2030  
Goal Target Indicator SeriesCode  GeoAreaName        
1    1.1    1.1.1     SI_POV_DAY1 Albania       NaN  
                                  Algeria       NaN  
                                  Angola        NaN  
                                  Argentina     NaN  
                                  Armenia       NaN

In [3]:
#hierarchical dataframe GeoAreaName_Indicator_SeriesCode (gis)
gis=sdg.set_index(['GeoAreaName','Goal','Target','Indicator','SeriesCode'],drop=True)
#pivot table where the columns are the years
gis=gis.pivot_table(index=['GeoAreaName','Goal','Target','Indicator','SeriesCode'], columns='TimePeriod', values='Value')

#subset columns 2005-2016
gis=gis[gis.columns[5:17]]

#count the null values in each row and output to a list
null_count=gis.notnull().sum(axis=1).tolist()

gis['NotNull']= array(null_count) / len(gis.columns)  #divide null_count by 12
gis.head(10)

#add 2 columns
gis['2025'] = numpy.nan
gis['2030'] = numpy.nan
gis.head()

TimePeriod                                                     2005  2006  \
GeoAreaName           Goal Target Indicator SeriesCode                      
 Iraq (Central Iraq)  16   16.1   16.1.1    VC_IHR_PSRC         NaN   NaN   
                                            VC_IHR_PSRCN        NaN   NaN   
Afghanistan           1    1.1    1.1.1     SI_POV_EMP1   81.266667   NaN   
                           1.2    1.2.1     SI_POV_NAHC         NaN   NaN   
                           1.3    1.3.1     SI_COV_PENSN        NaN   NaN   

TimePeriod                                                2007  2008  2009  \
GeoAreaName           Goal Target Indicator SeriesCode                       
 Iraq (Central Iraq)  16   16.1   16.1.1    VC_IHR_PSRC    NaN   NaN   NaN   
                                            VC_IHR_PSRCN   NaN   NaN   NaN   
Afghanistan           1    1.1    1.1.1     SI_POV_EMP1    NaN   NaN   NaN   
                           1.2    1.2.1     SI_POV_NAHC   36.3   NaN   NaN   
                           1.3    1.3.1     SI_COV_PENSN   NaN   NaN   NaN   

TimePeriod                                                   2010     2011  \
GeoAreaName           Goal Target Indicator SeriesCode                       
 Iraq (Central Iraq)  16   16.1   16.1.1    VC_IHR_PSRC      9.71     9.28   
                                            VC_IHR_PSRCN  2553.00  2518.00   
Afghanistan           1    1.1    1.1.1     SI_POV_EMP1       NaN      NaN   
                           1.2    1.2.1     SI_POV_NAHC       NaN    35.80   
                           1.3    1.3.1     SI_COV_PENSN    10.70      NaN   

TimePeriod                                                   2012     2013  \
GeoAreaName           Goal Target Indicator SeriesCode                       
 Iraq (Central Iraq)  16   16.1   16.1.1    VC_IHR_PSRC      9.24    11.11   
                                            VC_IHR_PSRCN  2594.00  3228.00   
Afghanistan           1    1.1    1.1.1     SI_POV_EMP1       NaN      NaN   
                           1.2    1.2.1     SI_POV_NAHC       NaN      NaN   
                           1.3    1.3.1     SI_COV_PENSN      NaN      NaN   

TimePeriod                                                   2014  2015  2016  \
GeoAreaName           Goal Target Indicator SeriesCode                          
 Iraq (Central Iraq)  16   16.1   16.1.1    VC_IHR_PSRC     10.08   NaN   NaN   
                                            VC_IHR_PSRCN  3029.00   NaN   NaN   
Afghanistan           1    1.1    1.1.1     SI_POV_EMP1       NaN   NaN   NaN   
                           1.2    1.2.1     SI_POV_NAHC       NaN   NaN   NaN   
                           1.3    1.3.1     SI_COV_PENSN      NaN   NaN   NaN   

TimePeriod                                                 NotNull  2025  2030  
GeoAreaName           Goal Target Indicator SeriesCode                          
 Iraq (Central Iraq)  16   16.1   16.1.1    VC_IHR_PSRC   0.416667   NaN   NaN  
                                            VC_IHR_PSRCN  0.416667   NaN   NaN  
Afghanistan           1    1.1    1.1.1     SI_POV_EMP1   0.083333   NaN   NaN  
                           1.2    1.2.1     SI_POV_NAHC   0.166667   NaN   NaN  
                           1.3    1.3.1     SI_COV_PENSN  0.083333   NaN   NaN

## Compare different regression models to predict for 2025 and 2030
This project only considers 2 models (linear regression and random forest). The following code can be modified to include more models and the best model is indicated by the highest `cross_val_score()`.

Regression models https://scikit-learn.org/stable/supervised_learning.html#supervised-learning

   - Examples https://scikit-learn.org/stable/auto_examples/index.html
    
Model selection and evaluation https://scikit-learn.org/stable/model_selection.html#model-selection

   - Cross validation  https://scikit-learn.org/stable/modules/cross_validation.html

In [299]:
import numpy as np
from sklearn.linear_model import LinearRegression #https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn.ensemble import RandomForestRegressor #https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
from sklearn.model_selection import cross_val_score
        #The `scoring` parameter https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
        #Regression metrics https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics

gis['ModelCV'] = numpy.nan
#for the ith row
for i in range(0,len(gis.values)):  
    row=gis.iloc[i][0:12] #columns 2005-2016
#y is the non-null columns; X is the corresponding years
    y=row[row.notnull()].values  #performance
    if len(y)<3:  #not predicting for 2025 and 2030 (leave as NaN) if there are <3 data entries
        continue
    X=array(row[row.notnull()].index).astype(int)  #year
    X=X.reshape(-1, 1)  #reshape(-1,1) because the data has only 1 predictor
#determin k to split the data into k groups (k-fold CV)
    if len(y)<6:  #3-5 data entries: make k as large as possible (k=n aka LOOCV)
        k=len(y)
    else:  #if >=6 data entries, split data into k=5 folds (a larger k means longer computing time)
        k=5

#LinearRegression() and k-fold cross validation 
    lr = LinearRegression().fit(X, y)
    #r2 = lr.score(X, y) #R^2 is defined as (1 - u/v), where u is the residual sum of squares and v is the total sum of squares
                        #Use the adjusted R-squared to compare models with different numbers of predictors

    #For regression, cross_val_score() uses scoring='r2'  by default 
        #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html#sklearn.metrics.r2_score
    #The score (R^2) for each fold
        #R^2 is a measure of how well future samples are likely to be predicted by the model
        #best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse)
    lr_cv_score= np.mean(cross_val_score(lr, X, y,cv=k))
        #another way to get the test_score is sklearn.model_selection.cross_validate(lm, X, y, cv=2,return_train_score=False)['test_score'] 

#LinearRegression() and k-fold cross validation 
    #random forest regression is able to capture non-linear interaction between the predictorand the response
    rf= RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
                            #max_depth restricts the tree depth to prevent overfitting
    rf.fit(X, y)
    rf_cv_score=np.mean(cross_val_score(rf, X, y,cv=k)) 

# input mean(cv_score) in column 'ModelCV' and predictions into columns '2025' and '2030'
    if lr_cv_score > rf_cv_score:  #linear regression is better
        gis.values[i][-1] = lr_cv_score
        gis.values[i][-2]= lr.predict(array(2030).reshape(-1, 1))
        gis.values[i][-3]= lr.predict(array(2025).reshape(-1, 1))
    else:
        gis.values[i][-1] = rf_cv_score
        gis.values[i][-2]= rf.predict(array(2030).reshape(-1, 1))
        gis.values[i][-3]= rf.predict(array(2025).reshape(-1, 1))

In [322]:
#Replace negative R^2 with 0 as negative R^2 means the  model fits the data poorly
for i in range(0,len(gis.values)):
    if gis.iloc[i]['ModelCV']<0:
        gis.iloc[i]['ModelCV']=0
gis.tail()

TimePeriod                                        2005   2006   2007   2008  \
GeoAreaName Goal Target Indicator SeriesCode                                  
Zimbabwe    9.0  9.4    9.4.1     EN_ATM_CO2GDP   0.49   0.47   0.49   0.46   
                                  EN_ATM_CO2MVA   1.40   2.06   2.28   1.39   
                 9.5    9.5.2     GB_POP_SCIERD    NaN    NaN    NaN    NaN   
                 9.a    9.a.1     DC_TOF_INFRAL   1.26   1.66   1.15   0.45   
                 9.c    9.c.1     IT_MOB_NTWK    52.50  62.00  64.00  65.05   

TimePeriod                                        2009   2010   2011   2012  \
GeoAreaName Goal Target Indicator SeriesCode                                  
Zimbabwe    9.0  9.4    9.4.1     EN_ATM_CO2GDP   0.42   0.49   0.52   0.49   
                                  EN_ATM_CO2MVA   0.46   0.99   1.03   1.05   
                 9.5    9.5.2     GB_POP_SCIERD    NaN    NaN    NaN  88.72   
                 9.a    9.a.1     DC_TOF_INFRAL   2.41  20.16   7.24   6.64   
                 9.c    9.c.1     IT_MOB_NTWK    41.00  33.00  43.70  58.00   

TimePeriod                                        2013       2014   2015  \
GeoAreaName Goal Target Indicator SeriesCode                               
Zimbabwe    9.0  9.4    9.4.1     EN_ATM_CO2GDP   0.48   0.450000   0.46   
                                  EN_ATM_CO2MVA   0.68   0.800000   0.73   
                 9.5    9.5.2     GB_POP_SCIERD    NaN        NaN    NaN   
                 9.a    9.a.1     DC_TOF_INFRAL  23.36  26.120000   6.39   
                 9.c    9.c.1     IT_MOB_NTWK    68.00  47.666667  51.60   

TimePeriod                                            2016   NotNull  \
GeoAreaName Goal Target Indicator SeriesCode                           
Zimbabwe    9.0  9.4    9.4.1     EN_ATM_CO2GDP        NaN  0.916667   
                                  EN_ATM_CO2MVA        NaN  0.916667   
                 9.5    9.5.2     GB_POP_SCIERD        NaN  0.083333   
                 9.a    9.a.1     DC_TOF_INFRAL  11.030000  1.000000   
                 9.c    9.c.1     IT_MOB_NTWK    55.596667  1.000000   

TimePeriod                                            2025       2030  ModelCV  
GeoAreaName Goal Target Indicator SeriesCode                                    
Zimbabwe    9.0  9.4    9.4.1     EN_ATM_CO2GDP   0.463426   0.463426      0.0  
                                  EN_ATM_CO2MVA   0.828644   0.828644      0.0  
                 9.5    9.5.2     GB_POP_SCIERD        NaN        NaN      NaN  
                 9.a    9.a.1     DC_TOF_INFRAL  31.253765  38.931212      0.0  
                 9.c    9.c.1     IT_MOB_NTWK    55.034270  55.034270      0.0

In [508]:
#update isg by unstacking-stacking gis
isg=gis.unstack(level=0).stack(level=-1)
isg.head()

TimePeriod                                     2005  2006  2007  2008  2009  \
Goal Target Indicator SeriesCode  GeoAreaName                                 
1.0  1.1    1.1.1     SI_POV_DAY1 Albania       1.1   NaN   NaN   0.4   NaN   
                                  Algeria       NaN   NaN   NaN   NaN   NaN   
                                  Angola        NaN   NaN   NaN  30.1   NaN   
                                  Argentina     3.9   3.3   2.9   2.6   2.6   
                                  Armenia       4.4   3.2   2.8   1.4   1.9   

TimePeriod                                     2010  2011  2012  2013  2014  \
Goal Target Indicator SeriesCode  GeoAreaName                                 
1.0  1.1    1.1.1     SI_POV_DAY1 Albania       NaN   NaN   1.1   NaN   NaN   
                                  Algeria       NaN   0.5   NaN   NaN   NaN   
                                  Angola        NaN   NaN   NaN   NaN   NaN   
                                  Argentina     1.1   0.9   0.8   0.8   0.7   
                                  Armenia       1.9   2.2   1.6   2.2   2.3   

TimePeriod                                     2015  2016   NotNull      2025  \
Goal Target Indicator SeriesCode  GeoAreaName                                   
1.0  1.1    1.1.1     SI_POV_DAY1 Albania       NaN   NaN  0.250000  0.904000   
                                  Algeria       NaN   NaN  0.083333       NaN   
                                  Angola        NaN   NaN  0.083333       NaN   
                                  Argentina     NaN   0.6  0.916667  0.705325   
                                  Armenia       1.9   1.8  1.000000  1.962310   

TimePeriod                                         2030  ModelCV  
Goal Target Indicator SeriesCode  GeoAreaName                     
1.0  1.1    1.1.1     SI_POV_DAY1 Albania      0.904000      0.0  
                                  Algeria           NaN      NaN  
                                  Angola            NaN      NaN  
                                  Argentina    0.705325      0.0  
                                  Armenia      1.962310      0.0

Note: another method to compare different models is to split X,y into training and testing sets. But for our dataset, it's better to use k-fold CV to compare models.
```python
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  #random_state is the seed
lm=LinearRegression()
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)
model.score(X_test, y_test) #R^2
```

In [ ]:
#back-up as csv files
isg.to_csv("C:\\Users\\Jiachen\\Downloads\\SDG Database UN\\results\\isg.csv")
gis.to_csv("C:\\Users\\Jiachen\\Downloads\\SDG Database UN\\results\\gis.csv")
#read saved work
isg=pd.read_csv("C:\\Users\\Jiachen\\Downloads\\SDG Database UN\\results\\isg.csv")
isg=isg.set_index(['Goal','Target','Indicator','SeriesCode','GeoAreaName'],drop=True)
gis=pd.read_csv("C:\\Users\\Jiachen\\Downloads\\SDG Database UN\\results\\gis.csv")
gis=gis.set_index(['GeoAreaName','Goal','Target','Indicator','SeriesCode'],drop=True)

## Standardizing data to 0-100
For each `{'Indicator','SeriesCode'}`, scale the `Value` to performance scores 0-100 with 100 indicating best performance. If a high `Value` means a poor performance towards an SDG, we mutiply the `Value` by -1 and then scale it to the performance score.

In `indicatorTypes.csv`, I identified different types of Indicators.
```
# hb: the higher the better 
# lb: the lower the better 
# b1: binary data with 1 being desirable
# b0: binary data with 0 being desirable
# ch: categorical value indicated by a number, the higher the better
# cl: categorical value indicated by a number, the lower the better
# ex: exclude the indicator for our analysis
```

In [11]:
indTypes = pd.read_csv("C:\\Users\\Jiachen\\OneDrive\\MDS Labs TEMP\\SDG\\generated data\\indicatorTypes.csv")
indTypes.head()

,Indicator,SeriesCode,IndicatorType
0,1.1.1,SI_POV_EMP1,lb
1,1.1.1,SI_POV_DAY1,lb
2,1.2.1,SI_POV_NAHC,lb
3,1.3.1,SI_COV_CHLD,hb
4,1.3.1,SI_COV_WKINJRY,hb


In [144]:
#flatten isg and left_join indTypes
tmp=isg.reset_index()
#move column NotNull to the right so the years 2005-2030 can be sliced
tmp=tmp[['Goal', 'Target', 'Indicator', 'SeriesCode', 'GeoAreaName', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2025', '2030', 'ModelCV', 'NotNull']]

tmp=pd.merge(tmp,indTypes, how='left', on=['Indicator','SeriesCode'])
print(sum(tmp.IndicatorType.isnull()), "rows where IndicatorType is null")
#drop rows where IndicatorType is 'ex'
tmp = tmp[tmp.IndicatorType != 'ex' ]

tmp=tmp.set_index(['Goal', 'Target', 'Indicator', 'SeriesCode','IndicatorType', 'GeoAreaName'])

#Standardize
import warnings
warnings.filterwarnings('ignore')  #don't show warnings
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 100))

#for each {'Indicator', 'SeriesCode'} level, standardize all countries' past and predicted performance to 0-100 
for idx, df in tmp.groupby(level=['Goal', 'Target', 'Indicator', 'SeriesCode','IndicatorType']):
    data = df.iloc[:,0:14]  #columns 2005-2030  
    if idx[-1] in ['lb', 'b0','cl']:   #IndicatorType is lb, b0, or cl
        data= -1 * data
    scaled_data = min_max_scaler.fit_transform(data)
    cols=['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014','2015', '2016', '2025', '2030']
    tmp.loc[idx,cols] = pd.DataFrame(scaled_data, data.index, data.columns)

0 rows where IndicatorType is null


In [161]:
tmp.describe()

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2025,2030,ModelCV,NotNull
count,24588.000000,21781.000000,21324.000000,21515.000000,22775.000000,27304.000000,23589.000000,24479.000000,22263.000000,22959.000000,27382.000000,22784.000000,29660.000000,29660.000000,29660.000000,49819.000000
mean,57.997553,60.577373,60.560584,61.532078,60.862295,57.242133,61.392194,59.066486,59.780085,59.765286,54.165950,53.883588,50.908285,48.699444,0.135262,0.472950
std,40.933783,40.582208,40.769030,41.229473,40.490597,40.895613,40.251944,41.069169,40.778685,40.782527,40.502646,41.029289,36.751654,35.046221,0.325353,0.399751
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.787899,15.384615,12.500000,11.778352,15.196832,8.727459,14.406158,10.385840,10.446722,10.719438,5.993639,3.506368,10.908734,11.447259,0.000000,0.083333
50%,71.983683,79.452055,80.565412,83.852798,80.120482,69.926332,79.797980,76.811594,77.074543,77.459016,61.807881,63.513514,57.291895,53.939087,0.000000,0.333333
75%,99.268479,99.622618,99.300000,99.710439,99.400196,99.000000,99.424887,99.300442,99.162993,99.405358,96.916300,97.434033,85.878825,79.441778,0.000000,0.916667
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000,1.000000


In [166]:
#set index for isg and gis (hierarchical)
isg=tmp.reset_index().drop(columns='IndicatorType').set_index(['Goal','Target','Indicator','SeriesCode','GeoAreaName'],drop=True)
gis=tmp.reset_index().drop(columns='IndicatorType').set_index(['GeoAreaName','Goal','Target','Indicator','SeriesCode'],drop=True)
#back-up as csv files
isg.to_csv("C:\\Users\\Jiachen\\Downloads\\SDG Database UN\\results\\isg.csv")
gis.to_csv("C:\\Users\\Jiachen\\Downloads\\SDG Database UN\\results\\gis.csv")

## Aggregation

In [172]:
#.mean() excludes missing values
AreaIndicator = gis.groupby(level=['GeoAreaName','Goal', 'Target', 'Indicator']).mean()
AreaTarget = AreaIndicator.groupby(level=['GeoAreaName','Goal', 'Target']).mean()
AreaGoal = AreaTarget.groupby(level=['GeoAreaName','Goal']).mean()

In [177]:
AreaGoal.head()

2005       2006       2007       2008  \
GeoAreaName           Goal                                               
 Iraq (Central Iraq)  16          NaN        NaN        NaN        NaN   
Afghanistan           1     15.022656        NaN  28.225047        NaN   
                      2     15.325530  24.796709  27.044214  36.465879   
                      3     47.963327  42.118812  54.263431  46.435613   
                      4           NaN   0.084527   0.067466   0.121064   

                                 2009       2010       2011       2012  \
GeoAreaName           Goal                                               
 Iraq (Central Iraq)  16          NaN  90.873729  91.927678  92.077090   
Afghanistan           1           NaN  34.355252  38.427969  27.611940   
                      2     37.587317  29.066893  37.427841  28.594972   
                      3     52.713816  47.837146  42.231484  44.226663   
                      4      0.271559   0.313120   0.368157   0.710774   

                                 2013       2014       2015       2016  \
GeoAreaName           Goal                                               
 Iraq (Central Iraq)  16    89.542514  89.811035        NaN        NaN   
Afghanistan           1     24.598744  48.217261  24.510188        NaN   
                      2     37.453303  29.305961  42.155052  38.305953   
                      3     50.419472  45.379009  43.976903  42.971814   
                      4      1.275766   0.691747   0.626004   0.541055   

                                 2025       2030   ModelCV   NotNull  
GeoAreaName           Goal                                            
 Iraq (Central Iraq)  16    89.038760  85.667573  0.000000  0.416667  
Afghanistan           1     41.571780  48.682361  0.000000  0.206667  
                      2     31.135574  31.803304  0.166667  0.390278  
                      3     51.100304  50.336473  0.060978  0.558681  
                      4      0.657303   0.748851  0.000000  0.916667

In [179]:
AreaTarget.to_csv("C:\\Users\\Jiachen\\Downloads\\SDG Database UN\\results\\AreaTarget.csv")
AreaGoal.to_csv("C:\\Users\\Jiachen\\Downloads\\SDG Database UN\\results\\AreaGoal.csv")